In [2]:
import pandas as pd
from sklearn.metrics import balanced_accuracy_score

In [3]:
from bias_explorer.utils import dataloader, system
from bias_explorer.operations import report

In [4]:
conf = dataloader.load_config()

In [5]:
df = dataloader.load_df(
    df_path="/home/lucasmc/Documents/ufrgs/vlms_bias_explorer/data/fairface/results/openCLIP/ViT-L-14/openai/top_synms.csv")

In [6]:
rep = dataloader.load_df(
    "/home/lucasmc/Documents/ufrgs/vlms_bias_explorer/data/fairface/reports/openCLIP/ViT-L-14/openai/raw_gender_labels/accuracy_report.csv")

In [7]:
rep.head()

,Unnamed: 0,Mode,accuracy,Male,Female,East Asian,White,Latino_Hispanic,Southeast Asian,Black,...,Middle Eastern,0-2,3-9,10-19,20-29,30-39,40-49,50-59,60-69,more than 70
0,0,Avg Sum,0.528757,1.0,0.0,0.50129,0.538129,0.488601,0.519435,0.513496,...,0.672457,0.61809,0.54646,0.46486,0.444848,0.56309,0.617886,0.623116,0.64486,0.508475
1,1,Top 1,0.528757,1.0,0.0,0.50129,0.538129,0.488601,0.519435,0.513496,...,0.672457,0.61809,0.54646,0.46486,0.444848,0.56309,0.617886,0.623116,0.64486,0.508475


In [8]:
df.head()

,file,age,gender,race,gender_preds,synm
0,val/1.jpg,3-9,Male,East Asian,Male,a photo of a man
1,val/2.jpg,50-59,Female,East Asian,Male,a photo of a man
2,val/3.jpg,30-39,Male,White,Male,a photo of a man
3,val/4.jpg,20-29,Female,Latino_Hispanic,Male,a photo of a woman
4,val/5.jpg,20-29,Male,Southeast Asian,Male,a photo of a man


In [9]:
white_df = report.filter_df(df, 'race', 'White')
print(report.gender_eval(df, balanced_accuracy_score))

0.5


In [10]:
prompts, _ = dataloader.load_txts(conf['Labels'])

In [11]:
prompts

['a photo of a man', 'a photo of a woman']

In [13]:
if "woman" in prompts[0]:
    print("woman found")